# Programming exercise with the radiative transfer model RRTMG
# METRAD module, University of Cologne

*Questions:* kerstin.ebell@uni-koeln.de, a.walbroel@uni-koeln.de

The energetic impact of atmospheric radiation will be analyzed using the broadband, one-dimensional radiative transfer model RRTMG of the Atmospheric and Environmental Research, Incorporated ([RRTM](http://rtweb.aer.com/rrtm_frame.html)). Here, we use RRTMG in the python framework pyRRTMG, which is used by [T-CARS](https://doi.org/10.5281/zenodo.11147087)  (Tropospheric Research Cloud and Aerosol Radiative effect Simulator). RRTMG is implemented in many numerical weather prediction and climate models, e.g. in the ECMWF Integrated Forecasting System, the NCEP Global Forecast System, the climate model ECHAM5 and in the operational global model ICON of the Deutscher Wetterdienst.

RRTMG consists of 14 consecutive spectral bands in the solar spectral region (UV/VIS: 0.2-0.625 μm; NIR: 0.625-12.195 μm) and 16 bands in the longwave spectral region (3.077-1000 μm). It accounts for absorption of longwave radiation and extinction, i.e. absorption and scattering, of shortwave radiation by water vapor (H<sub>2</sub>O), carbon dioxide (CO<sub>2</sub>), ozone (O<sub>3</sub>), methane (CH<sub>4</sub>), molecular oxygen (O<sub>2</sub>), nitrous oxide (N<sub>2</sub>O), liquid water and ice clouds. The effect of clouds is taken into account by means of their liquid and ice water content, the corresponding effective radii and cloud fraction. In addition, assumptions on the overlapping of clouds at different heights need to be made. Usually, the so called maximum-random overlap is assumed. This means, that clouds of adjacent height levels maximally overlap and randomly, if they are separated by cloud-free layers.

Multiple scattering of radiation is allowed for in the RRTMG by the two-stream approach: instead of calculating radiances coming from each solid angle, only irradiances (fluxes) from two directions, i.e. a downward component from the upper hemisphere and a upward component of the lower hemisphere, are considered. This approach is usually applied in numerical weather prediction and climate models, since it enables a quite accurate calculation of irradiances and thus atmospheric heating rates

### Preparation

1. (If needed:) Copy some background data needed for T-CARS: subarctic_summer.txt to .../vmr_Anderson_1986/ and NOAA_Annual_Mean_MoleFractions_2023.csv to .../trace_gas_data/. Then, change `path_data` in the **metrad_rrtmg_ex.py** cell in such a way that the path points to the directory where the folders vmr_Anderson_1986 and trace_gas_data are located.
2. While answering the questions, you have to change some parameters in the **metrad_rrtmg_ex.py** cell. Also make sure to rename your output files and plots for each task. Otherwise, your previous results will be overwritten! For example, you can use and set the `suffix` and `subtask` variables in the **metrad_rrtmg_ex.py** cell.
3. Run all cells step by step. The first couple of cells only define functions and constants used for the RRTMG simulations and plotting.
4. You may save the simulation results using `tcars_client.save_output`. The command `ncdump -h <file>` shows which variables have been stored:
   | Name | Description | Unit |
   | ---- | ----------- | ---- |
   | time | not important here, undefined | / |
   | height | height of the 26 full level (center of layer) | in m |
   | height_h | height of the 27 half level (layer boundaries) | in m |
   | swuflx | Upward shortwave flux at the 27 half levels | in W m-2 |
   | swdflx | Downward shortwave flux | in W m-2 |
   | swdirflx | Direct shortwave flux | in W m-1 |
   | swhr | Shortwave radiative heating rates for layers | in K day-1 |
   | swuflxc | Clear sky upward shortwave flux | in W m-2 |
   | swdflxc | Clear sky downward shortwave flux | in W m-2 |
   | swdirflxc | Clear sky direct shortwave flux | in W m-2 |
   | swhrc | Clear sky shortwave radiative heating rates | in K day-1 |
   | lwuflx | Upward longwave flux | in W m-2 |
   | lwdflx | Downward longwave flux | in W m-2 |
   | lwhr | Longwave radiative heating rates | in K day-1 |
   | lwuflxc | Clear sky upward longwave flux | in W m-2 |
   | lwdflxc | Clear sky downward longwave flux | in W m-2 |
   | lwhrc | Clear sky longwave radiative heating rates | in K day-1 |

The radiative fluxes and heating rates are calculated for one atmospheric profile (mid-latitude summer, MLS), which consists of 26 vertical layers with a thickness of 1 km. The radiative fluxes are provided for the layer boundaries, the heating rates for the model layers. When the radiation budget is positive (more radiation goes into the layer than out of the layer), the layer is heated. In the opposite case, the atmospheric layer is cooled.

Below, necessary constants and functions used for this exercise are defined and loaded. For the exercise, only the **metrad_rrtmg_ex.py** cell must be modified.

In [ ]:
import os
import sys
import pdb
from tqdm import tqdm
from collections import OrderedDict as odict

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import csv
import pandas as pd

import pyrrtmg as rrtmg


In [ ]:
# constants.py

R_d = 287.0597  # gas constant of dry air, in J kg-1 K-1
R_v = 461.5     # gas constant of water vapour, in J kg-1 K-1
R_ = 8.314462618    # universal gas constant, in J mol-1 K-1, https://physics.nist.gov/cgi-bin/cuu/Value?r
M_dv = R_d / R_v # molar mass ratio , in ()

m_mol_air = 0.0289647       # molar mass of dry air, in kg mol-1, https://www.engineeringtoolbox.com/molecular-mass-air-d_679.html
mw_h2o = 0.01802  # h2o molar mass in kg mol-1
mw_o3 = 0.0479982 # ozone molar mass in kg mol-1
mw_co2 = 0.0440   # co2 molar mass in kg mol-1
mw_n2o = 0.01802  # n2o molar mass in kg mol-1
mw_co = 0.02801   # co molar mass in kg mol-1
mw_ch4 = 0.01604  # ch4 molar mass in kg mol-1
mw_o2 = 0.015999  # o2 molar mass in kg mol-1

e_0 = 611       # saturation water vapour pressure at freezing point (273.15 K), in Pa
T0 = 273.15     # freezing temperature, in K
c_pd = 1005.7   # specific heat capacity of dry air at constant pressure, in J kg-1 K-1
c_vd = 719.0    # specific heat capacity of dry air at constant volume, in J kg-1 K-1
c_h2o = 4187.0  # specific heat capacity of water at 15 deg C; in J kg-1 K-1
L_v = 2.501e+06 # latent heat of vaporization, in J kg-1
sigma_sb = 5.670374419e-08  # Stefan-Boltzmann constant W m-2 K-4, 
                            # https://physics.nist.gov/cgi-bin/cuu/Value?sigma|search_for=stefan

g = 9.80665     # gravitation acceleration, in m s^-2 (from https://doi.org/10.6028/NIST.SP.330-2019 )
omega_earth = 2*np.pi / 86164.09    # earth's angular velocity: World Book Encyclopedia Vol 6. Illinois: World Book Inc.: 1984: 12.
R_e = 6371000   # volumetric radius of earth in m, https://nssdc.gsfc.nasa.gov/planetary/factsheet/earthfact.html

sfac = {'ppm': 1e-06, 'ppb': 1e-09, 'ppt': 1e-12}   # acronyms, scaling factor for ppm, ppb, ...

solar_constant = 1368.      # Solar constant, in W m-2 ; 
                            # some sources suggest 1367 W m-2 (https://doi.org/10.1016/B978-0-08-087872-0.00302-4)

In [ ]:
# data_tools.py

def encode_time(DS: xr.Dataset):
    DS['time'] = DS['time'].values.astype("datetime64[s]").astype(np.float64)
    DS['time'].attrs['units'] = "seconds since 1970-01-01 00:00:00"
    DS['time'].encoding['units'] = 'seconds since 1970-01-01 00:00:00'
    DS['time'].encoding['dtype'] = 'double'
    
    return DS


In [ ]:
# met_tools.py

def q_to_h2ovmr(q: np.ndarray):
    
    """
    Converts specific humidity q (in kg kg-1) to volume mixing ratio (unitless).
    
    Parameters:
    -----------
    q : np.ndarray or xr.DataArray
        Specific humidity in kg kg-1.
    """
    
    return m_mol_air*q / (mw_h2o*(1.0 - q))


def h2ovmr_to_q(h2ovmr: np.ndarray):
    
    """
    Converts water vapour volume mixing ratio (unitless) to specific humidity q (in kg kg-1).
    
    Parameters:
    -----------
    h2ovmr : np.ndarray or xr.DataArray
        Water vapour volume mixing ratio.
    """
    
    return h2ovmr / ((m_mol_air/mw_h2o) + h2ovmr)


def rho_air(
    pres,
    temp,
    abshum):

    """
    Compute the density of air (in kg m-3) with a certain moisture load.

    Parameters:
    -----------
    pres : array of floats
        Array of pressure (in Pa).
    temp : array of floats
        Array of temperature (in K).
    abshum : array of floats
        Array of absolute humidity (in kg m^-3).
    """

    rho = (pres - abshum*R_v*temp) / (R_d*temp) + abshum

    return rho


def convert_spechum_to_abshum(
    temp,
    pres,
    q):

    """
    Convert array of specific humidity (kg kg^-1) to absolute humidity
    in kg m^-3.

    Parameters:
    -----------
    temp : array of floats
        Array of temperature (in K).
    pres : array of floats
        Array of pressure (in Pa).
    q : array of floats
        Array of specific humidity (in kg kg^-1).
    """

    abshum = pres / (R_d*temp*(1/q + 1/M_dv - 1))

    return abshum


def compute_heating_rate(
    upward_flux: np.ndarray,
    downward_flux: np.ndarray,
    rho: np.ndarray,
    height_lev: np.ndarray,
    convert_to_K_day=False):
    
    """
    Compute heating rates according to Petty (2006) chapter 10.4.1, equation 10.54 from
    upward and downward radiation fluxes (shortwave and longwave possible). Note that
    the radiation fluxes must be given on height levels while air density must be provided 
    on a height layer grid (whose boundaries are the height levels) because the heating rates
    will be put onto the height layer grid.
    
    Parameters:
    -----------
    upward_flux : np.ndarray or xr.DataArray
        Upward shortwave or longwave radiation flux in W m-2.
    downward_flux : np.ndarray or xr.DataArray
        Downward shortwave or longwave radiation flux in W m-2.
    rho : np.ndarray or xr.DataArray
        Air density (dry air + absolute humidity) in kg m-3.
    height_lev : np.ndarray or xr.DataArray
        Height levels (boundaries of height layers) in m.
    convert_to_K_day : bool
        If True, heating rates will be given in K day-1. Otherwise, in K s-1.
    """
    
    F_net = upward_flux - downward_flux
    dF_net_dz = np.diff(F_net, axis=-1) / np.diff(height_lev, axis=-1)
    HR = - dF_net_dz / (c_pd * rho)       # heating rate in K s-1
    if convert_to_K_day: HR *= 86400.               # heating rate in K day-1
    
    return HR

In [ ]:
# plot_styles.py

linewidth = 2
panel_marker = ["a","b","c","d","e","f","g"]

In [ ]:
# readers_writers.py

def read_default_profiles(scaling=None, offset=None):
    
    
    def apply_offset_and_scaling(data_dict: dict, scaling: dict, offset: dict):
        
        for var in data_dict.keys():
            scaling_ = 1.
            offset_ = 0.
            if var in scaling.keys():
                scaling_ = scaling[var]
            if var in offset.keys():
                offset_ = offset[var]
            
            data_dict[var] = (data_dict[var] + offset_) * scaling_
            
        return data_dict
    
    
    data_dict = dict(
        pres=np.array([955.8902, 850.5316, 754.5992, 667.7425, 589.8407, 519.4208, 455.4800,
                       398.0854, 347.1714, 301.7350, 261.3102, 225.3597, 193.4192, 165.4902, 141.0319,
                       120.1249, 102.6889, 87.8294, 75.1226, 64.3059, 55.0863, 47.2091, 40.5354, 34.7954,
                       29.8654, 22.9835]),     # pressure at full levels, in hPa
    
        pres_h=np.array([1013.000, 902.000, 802.000, 710.000, 628.000, 554.000, 487.000, 426.000, 
                         372.000, 324.000, 281.000, 243.000, 209.000, 179.000, 153.000, 130.000, 111.000, 
                         95.000, 81.200, 69.500, 59.500, 51.000, 43.700, 37.600, 32.200, 27.700, 19.070]),
                         # pressure at half levels, in hPa
        
        temp_h=np.array([294.200, 289.700, 285.200, 279.200, 273.200, 267.200, 261.200, 254.700, 
                         248.200, 241.700, 235.300, 228.800, 222.300, 215.800, 215.700, 215.700, 215.700,
                         215.700, 216.800, 217.900, 219.200, 220.400, 221.600, 222.800, 
                         223.900, 225.100, 228.450]), # temperature at half levels, in K
        
        height_h=np.array([0., 1000., 2000., 3000., 4000., 5000., 6000., 7000., 
                           8000., 9000.,10000., 11000., 12000., 13000., 14000., 15000., 
                           16000., 17000., 18000., 19000., 20000., 21000., 22000., 
                           23000., 24000., 25000., 27500.]),
                           # auxiliary half level heights, in m

        temp_sfc=294.2,        # temperature at surface, in K
        lwp=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0]),    # in-cloud lwp, in g m-2
        iwp=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                      0, 0]),    # in-cloud iwp, g m-2
        clc=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                      0, 0]),    # cloud cover
        re_liq=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0]), # effecive radius, in um
        re_ice=np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 
                         0, 0]), # effective radius ice, in um
        h2o_vmr=np.array([0.0101199, 0.0072961, 0.0048715, 0.0030469, 0.0018778, 
                          0.0011616, 0.0007868, 0.0005183, 0.0003294, 0.0002053, 0.0001066, 
                          0.0000389, 0.0000116, 0.0000040, 0.0000026, 0.0000021, 0.0000020, 
                          0.0000020, 0.0000020, 0.0000020, 0.0000021, 0.0000022, 0.0000023, 
                          0.0000024, 0.0000026, 0.0000027]),   
                          # is actually specific humidity in kg kg-1 --> is converted to vmr below

        co2_vmr=280*sfac['ppm'],
        o3_vmr=0.,
        n2o_vmr=0.32*sfac['ppm'],
        ch4_vmr=1.60*sfac['ppm'],
        o2_vmr=209480*sfac['ppm'],
        
        julian_day=251,
        cos_zenith=0.7,        # cosine of zenith angle
        alb_dir_uv=0.2,        # direct albedo UV spectrum
        alb_dif_uv=0.2,        # diffuse albedo UV spectrum
        alb_dir_nir=0.2,       # direct albedo near IR spectrum
        alb_dif_nir=0.2,       # diffuse albedo near IR spectrum
        
        # tau_aero_sw=0.,      # aerosol optical thickness UV spectrum
        # ssa_aero_sw=0.,      # single scattering albedo UV spectrum
        # asm_aero_sw=0.,      # asymmetry parameter UV spectrum
        # tau_aero_lw=0.,      # aerosol optical thickness IR spectrum
        # emissivity=40*0.996,
        )
    
    for var in ['temp', 'height']:
        data_dict[var] = 0.5*(data_dict[var+"_h"][...,:-1] + data_dict[var+"_h"][...,1:])
    
    data_dict = apply_offset_and_scaling(data_dict, scaling, offset)
    
    data_dict['h2o_vmr'] = q_to_h2ovmr(data_dict['h2o_vmr'])    # kg kg-1 to volume mixing ratio
    time = pd.to_datetime(data_dict['julian_day'], unit="D")
    data_dict['time'] = np.datetime64(pd.Timestamp(f"2020-{time.month}-{time.day}T00:00:00"))
    
    DS = xr.Dataset(coords={'time': (['time'], np.array([data_dict['time']]).astype('datetime64[ns]')),
                            'height': (['height'], data_dict['height']),
                            'height_h': (['height_h'], data_dict['height_h'])})
    n_time, n_hgt, n_hgt_h = len(DS.time), len(DS.height), len(DS.height_h)
    
    gas_list = ['h2o_vmr', 'co2_vmr', 'o3_vmr', 'n2o_vmr', 'ch4_vmr', 'o2_vmr']
    for var in data_dict.keys():
        if var in DS.coords: continue
        
        if isinstance(data_dict[var], (float, int)) and (var in gas_list):
            DS[var] = xr.DataArray(np.broadcast_to(np.array([data_dict[var]]), (n_time, n_hgt)), 
                                   dims=['time', 'height'])
            
        elif isinstance(data_dict[var], (float, int)) and (var not in gas_list):
            DS[var] = xr.DataArray(np.array([data_dict[var]]), dims=['time'])
            
        else:
            var_shape = data_dict[var].shape
            if (n_hgt in var_shape) & (n_time not in var_shape):
                DS[var] = xr.DataArray(np.reshape(data_dict[var], (n_time, n_hgt)), dims=['time', 'height'])
            elif (n_hgt_h in var_shape) & (n_time not in var_shape):
                DS[var] = xr.DataArray(np.reshape(data_dict[var], (n_time, n_hgt_h)), dims=['time', 'height_h'])
            
    return DS
    
    
def import_vmr_std_atm(file: str):
    
    """
    Imports greenhouse gas concentrations from standard atmospheres given by
    Anderson, G P, Clough, S A, Kneizys, F X, Chetwynd, J H, and Shettle, E P. AFGL (Air Force 
    Geophysical Laboratory) atmospheric constituent profiles (0. 120km). Environmental research 
    papers. United States: N. p., 1986. Web.
    Converts to volumne mixing ratios.
    
    Parameters:
    -----------
    file : str
        Full path to additional T-CARS data.
    """
    
    df = pd.read_table(file, sep='\s+',
                       names=('height','PPP','TTT','air','h2o','co2','o3','n2o','co','ch4','o2'),
                       dtype={'height': np.float32,'PPP': np.float32,'TTT': np.float32,'air': np.float32,
                              'h2o': np.float32,'co2': np.float32,'o3': np.float32,'n2o': np.float32,
                              'co': np.float32,'ch4':np.float32,'o2':np.float32}, skiprows=1)
    df.index = df['height']

    DS = xr.Dataset(coords={'height': (['height'], df.index.values*1000.)})
    for var in df.columns:
        if var in DS.coords: 
            continue
        elif var in ['h2o', 'co2', 'o3', 'n2o', 'co', 'ch4', 'o2']:
            df[var] = sfac['ppm'] * df[var].values
        DS[var] = xr.DataArray(df[var].values, dims=['height'])
    
    return DS


def import_trace_gas_csv(file):

    """
    Imports the NOAA Trace Gas data from "https://gml.noaa.gov/aggi/aggi.html" and puts it
    into an xarray data set.

    Parameters:
    -----------
    file : str
        Full path and filename of trace gas concentration data in a .csv table.
    """

    # open the csv and read it:
    with open(file, newline='') as csvfile:

        csv_reader = csv.reader(csvfile, delimiter=',')
        list_of_lines = [row for row in csv_reader]

    # below the header, the trace gas name and units can be extracted:
    tg_species = [species.lower().replace('-','') for species in list_of_lines[2] if species != ""]
    units = [unit for unit in list_of_lines[3] if unit != ""]
    assert len(tg_species) == len(units)

    # extract data: multiply data with the respective unit
    data_lines = list_of_lines[4:-5]
    data = np.array([[np.nan if dd == "nd" else float(dd)*sfac[units[kk]] for kk, dd in enumerate(data_line[1:])] 
                    for data_line in data_lines])
    data_times = np.array([np.datetime64(f"{int(float(data_line[0]))}-07-01T00:00:00") for data_line in data_lines])

    # initialize data set and fill it with data
    DS = xr.Dataset(coords={'time': (['time'], data_times.astype('datetime64[ns]'))})
    for kk, species in enumerate(tg_species):
        DS[species] = xr.DataArray(data[:,kk], dims=['time'])

    return DS

In [ ]:
# tcars.py

class tcars:
    
    """
    Loads all necessary constants for radiative transfer simulations using T-CARS (python interface of RRTMG)*.
    
    *: Deneke, H. (2024) hdeneke/pyRRTMG: Release with correct versioning scheme . Zenodo [code]. https://doi.org/10.5281/zenodo.11147087
    
    Init:
    path_tcars_data : str
        Full path where additional data for T-CARS is stored.
    DS : xr.dataset
        Dataset containing atmospheric data. Must have "time" and "height" dimensions.
    """
    
    def __init__(self, path_tcars_data: str, DS: xr.Dataset):
        
        self.DS = DS
        self.path_tcars_data = path_tcars_data
        
        # for more detailed descriptions of the flags below, see 
        # https://github.com/hdeneke/pyRRTMG/blob/master/src_f/rrtmg_sw/column_model/doc/rrtmg_sw_instructions.txt
        self.icld = 1       # icld = 1 if clouds should be included; icld = 0 if clouds are omitted; 
                            # flag values 2 or 3 change the "overlap assumption"
        self.iaer = 0       # iaer = 10: one or more layers contain aerosols; = 0 to omit aerosols
        self.permuteseed_sw = 0     # permuteseed_sw
        self.permuteseed_lw = 0     # permuteseed_lw
        self.irng = 0               # irng
        self.idrv = 0               # idrv https://github.com/hdeneke/pyRRTMG/tree/master/src_f/rrtmg_lw -> 4th paragraph
                                    # --> whether to also calculate the derivative of flux with respect to surface temp
        
        self.adjes = 1.0        # adjustment of total solar irradiance (solar constant)
        self.solar_constant = solar_constant      # solar constant; total solar irradiance at TOA
        self.inflgsw = 2            # 0: direct specif. of cloud opt depth, cloud fraction, single scat albedo, phase function
                                    # 2: ice and liq cloud opt depths are calculated from lwp, iwp, effective radii
        self.inflglw = 2            # similar to inflgsw; see also INFLAG in pyRRTMG doc noted above
        self.iceflgsw = 2           # see ICEFLAG in pyRRTMG doc
        self.iceflglw = 2
        self.liqflgsw = 1           # see LIQFLAG in pyRRTMG doc
        self.liqflglw = 1
        
        # flags for the use of gas volume mixing ratios: 
        # -1: turn off, _vmr is set to 0
        # 0: use reference given in standard atmosphere (Anderson 1986)
        # 1: use values given in DS
        self.iflag_h2o_vmr = 1
        self.iflag_co2_vmr = 0      # additional option: 2: use NOAA measurements
        self.iflag_o3_vmr = 0
        self.iflag_n2o_vmr = 1      # additional option: 2: use NOAA measurements
        self.iflag_ch4_vmr = 1      # additional option: 2: use NOAA measurements
        self.iflag_o2_vmr = 1
        
        self.iflag_emissivity = 0   # -1: use emissivity of 1, 0: use emissivity of 0.996, 
                                    # 1: user-defined emissivity in set_emissivity
                
        self.set_translation_dict()
        self.init_cloud_properties()
        self.init_aerosol_properties()
        self.set_emissivity()
        
        
    def load_background_vmrs(self):
        
        DS = import_vmr_std_atm(self.path_tcars_data + "vmr_Anderson_1986/subarctic_summer.txt")
        DS = DS.expand_dims(dim='time', axis=0)
        DS = DS.interp(coords={'height': self.DS.height.values}, kwargs={"fill_value": 'extrapolate'})
        
        n_time, n_hgt = len(self.DS.time), len(self.DS.height)
        dims_list = ['time', 'height']
        for var in ['h2o', 'co2', 'o3', 'n2o', 'co', 'ch4', 'o2']:
            var_vmr = var + "_vmr"
            
            if (f'iflag_{var_vmr}' in self.__dict__.keys()) and (self.__dict__[f'iflag_{var_vmr}'] == 0):
                self.DS[var_vmr] = xr.DataArray(DS[var].values, dims=dims_list)
            elif (f'iflag_{var_vmr}' not in self.__dict__.keys()):
                self.DS[var_vmr] = xr.DataArray(DS[var].values, dims=dims_list)
        
        add_ghgs = {'cfc11': 232.0, 
                    'cfc12': 516.0, 
                    'cfc22': 233.0, 
                    'ccl4': 82.0}       # all in ppt, Source: NOAA
        for var in add_ghgs:
            self.DS[var+"_vmr"] = xr.DataArray(np.full((n_time, n_hgt), add_ghgs[var]*sfac['ppt']), 
                                               dims=dims_list)
        
        DS = import_trace_gas_csv(self.path_tcars_data + "/trace_gas_data/NOAA_Annual_Mean_MoleFractions_2023.csv")
        DS = DS.interp(time=self.DS.time.values).expand_dims(dim={'height': self.DS.height.values}, axis=1)

        for var in ['co2', 'n2o', 'ch4', 'cfc11', 'cfc12', 'ccl4']:
            var_vmr = var + "_vmr"
            if (f'iflag_{var_vmr}' in self.__dict__.keys()) and (self.__dict__[f'iflag_{var_vmr}'] == 2):
                self.DS[var_vmr] = xr.DataArray(DS[var].values, dims=dims_list)
            elif (f'iflag_{var_vmr}' not in self.__dict__.keys()):
                self.DS[var_vmr] = xr.DataArray(DS[var].values, dims=dims_list)

    
    def set_rrtmg_input(self):
        
        """
        Forward data loaded into Dataset to dictionaries used by T-CARS (=RRTMG).
        """

        self.load_background_vmrs()
        self.update_DS()
        self.update_cloud_properties()
        self.update_aerosol_properties()

        atm_args = ['Play', 'Plev',                                  # pressure of layer and level in hPa
                    'Tlay', 'Tlev', 'Tsfc',                          # temperature of layer and level in K
                    'h2ovmr', 'o3vmr', 'co2vmr', 'ch4vmr', 'n2ovmr', # vol mix ratio of gases
                    'o2vmr', 'cfc11vmr', 'cfc12vmr', 'ccl4vmr', 'cfc22vmr']
        
        atm = dict()
        for ds_var, t_var in self.trl_.items():
            if t_var in atm_args:
                try:
                    atm[t_var] = np.asfortranarray(self.DS[ds_var], dtype=np.float64)
                except KeyError:
                    print(f"{ds_var} is needed but was not found in the dataset provided to set_rrtmg_input.")
                
                
        for var in [
                    'alb_dir_uv',           # UV/VIS surface albedo, direct (0.2-0.625 um)
                    'alb_dif_uv',           # UV/VIS surface albedo, diffuse (0.2-0.625 um)
                    'alb_dir_nir',          # near-IR surface albedo, direct (0.625-12.20 um)
                    'alb_dif_nir',          # near-IR surface albedo, diffuse (0.625-12.20 um)
                    'julian_day',           # "Day of the year" or Julian Day number
                    'cos_zenith',           # cosine of the solar zenith angle
                    ]:
            
            if var in self.DS.data_vars:
                self.__setattr__(var, self.DS[var].values[0])
            
            # Fill values if data not available in DS:
            elif var == 'julian_day':
                self.julian_day = 1
            elif var in ["alb_dir_uv", "alb_dif_uv", "alb_dir_nir", "alb_dif_nir"]:
                self.__setattr__(var, 0.1)
            elif var == 'cos_zenith':
                self.cos_zenith = 0.0
        
        
        self.rrtmg_input = [self.icld,
                            self.iaer,
                            self.permuteseed_sw,
                            self.permuteseed_lw,
                            self.irng,
                            self.idrv]
        
        self.rrtmg_input += [atm[k] for k in atm_args]
        
        self.rrtmg_input += [self.alb_dif_nir, 
                             self.alb_dir_nir, 
                             self.alb_dif_uv, 
                             self.alb_dir_uv,
                             self.emis,
                             self.cos_zenith,
                             self.adjes,
                             self.julian_day,
                             self.solar_constant,
                             self.inflgsw,
                             self.inflglw,
                             self.iceflgsw,
                             self.iceflglw,
                             self.liqflgsw,
                             self.liqflglw]
        
        for var in self.cloud_props.keys():
            self.rrtmg_input += [self.cloud_props[var]]
        for var in self.aerosol_props.keys():
            self.rrtmg_input += [self.aerosol_props[var]]
        
        self.rrtmg_input = odict(zip(rrtmg.input_vars, self.rrtmg_input))
        
        
        for var in ['h2o', 'co2', 'o3', 'n2o', 'ch4', 'o2']:
            if self.__dict__[f'iflag_{var}_vmr'] == -1:
                self.rrtmg_input[var+"vmr"][...] = 0
    
    
    def set_translation_dict(self):
        
        """
        Translation dict to rename variables of self.DS (keys) to the names used by T-CARS 
        (values).
        """
        
        self.trl_ = {'pres': 'Play',
                     'temp': 'Tlay',
                     'temp_sfc': 'Tsfc',
                     'pres_h': 'Plev',
                     'temp_h': 'Tlev',
                     'lwp': 'lwp',
                     'iwp': 'iwp',
                     'clc': 'cldfrac',
                     're_liq': 're_liq',
                     're_ice': 're_ice',
                     'tauc_sw': 'tauc_sw',
                     'tauc_lw': 'lauc_lw',
                     'ssac_sw': 'ssac_sw',
                     'asmc_sw': 'asmc_sw',
                     'fsfc_sw': 'fsfc_sw',
                     }
        
        for var in ['h2o_vmr', 'co2_vmr', 'o3_vmr', 'n2o_vmr', 'ch4_vmr', 'o2_vmr',
                    'cfc11_vmr', 'cfc12_vmr', 'cfc22_vmr', 'ccl4_vmr']:
            self.trl_[var] = var.replace('_vmr', 'vmr')
    
    
    def init_cloud_properties(self):
        
        self.cloud_props = dict()
        for var in [
                    'tauc_sw',      # in cloud optical depth, short wave
                    'tauc_lw',      # in cloud optical depth, long wave
                    'cldfrac',      # cloud fraction
                    'ssac_sw',      # in cloud single scattering albedo
                    'asmc_sw',      # in cloud assymetry parameter
                    'fsfc_sw',      # in cloud forward scattering fraction, 
                                    # delta function pointing forward; forward peaked scattering
                    'iwp',          # cloud ice water path, in g m-2
                    'lwp',          # cloud liquid water path, in g m-2
                    're_ice',       # effective radius ice, in um
                    're_liq',       # effective radius liquid, in um
                    ]:
            
            shape_ = (len(self.DS.time), len(self.DS.height))
            fill_val = 0.0
            if "_sw" in var:
                shape_ = (rrtmg.nbnd_sw,) + shape_
            elif "_lw" in var:
                shape_ = (rrtmg.nbnd_lw,) + shape_
                
            self.cloud_props[var] = np.full(shape_, fill_val, dtype=np.float64, order='F')
    
    
    def init_aerosol_properties(self):
        
        shape_base = (len(self.DS.time), len(self.DS.height))
        
        kw = {'dtype': np.float64, 'order': "F"}
        self.aerosol_props = {
            "tauaer_sw": np.full(shape_base + (rrtmg.nbnd_sw,), 0.0, **kw),    
                         # aerosol opt depth (iaer=10 only); (ncol,nlay,nbndsw)
                         # iaer=10 means one or more layers contain aerosols
            "ssaaer_sw": np.full(shape_base + (rrtmg.nbnd_sw,), 1.0, **kw),    
                         # aerosol single scat albedo (iaer=10 only)
            "asmaer_sw": np.full(shape_base + (rrtmg.nbnd_sw,), 0.0, **kw),
                         # aerosol assymetry parameter (iaer=10 only)
            "ecaer_sw": np.full(shape_base + (6,), 0.0, **kw),
                        # aerosol opt depth at 0.55 um (iaer=6 only)
            "tauaer_lw": np.full(shape_base + (rrtmg.nbnd_lw,), 0.0, **kw),
                         # aerosol opt depth at mid-point of LW spectral bands
            }
    
    
    def update_DS(self):
        
        """
        Update temperature data on height layers and use temperature of lowest level as surface 
        temperature. This ensures consistency when editing temperature levels.
        """
        
        self.DS['temp'][...] = 0.5*(self.DS['temp_h'].values[...,:-1] + self.DS['temp_h'].values[...,1:])
        self.DS['temp_sfc'][:] = self.DS['temp_h'].sel(height_h=0., method='nearest')
    
    
    def update_cloud_properties(self):
        
        self.cloud_props = self.update_properties(self.cloud_props)
            
            
    def update_aerosol_properties(self):
        
        self.aerosol_props = self.update_properties(self.aerosol_props)
    
    
    def update_properties(self, props: dict):
        
        prop_args = [*props.keys()]
        for var in self.DS.data_vars:
            
            if var in prop_args:
                var_update = var
            elif (var in self.trl_.keys()) and (self.trl_[var] in prop_args):
                var_update = self.trl_[var]
            else:
                continue
            props[var_update] = self.DS[var].values
        
        return props
    
    
    def set_emissivity(self):
        
        """
        Emissivity: 16 bands in longwave, see 
        https://github.com/hdeneke/pyRRTMG/blob/master/src_f/rrtmg_lw/column_model/doc/rrtmg_lw_instructions.txt
        -> 3-1000 um
        
        Use 0.999 for sea ice fraction >= 0.5.
        """
        
        array_kwargs = dict(dtype=np.float64, order='F')
        emis_shape = (len(self.DS.time), 16)
        if self.iflag_emissivity == -1:
            emis_val = 1.0
        elif self.iflag_emissivity == 0:
            emis_val = 0.996
        elif self.iflag_emissivity == 1:
            # define your own emissivity array of emis_shape or use the following:
            emis_val = 0.9837
        self.emis = np.ones(emis_shape, **array_kwargs) * emis_val
    
    
    def run_tcars(self):
        
        self.run_sanity_test()
        self.flxhr = rrtmg.calc_flxhr(**self.rrtmg_input)
        self.organise_output()
        
        
    def run_sanity_test(self):
        
        for var in ['play', 'plev', 'tlay', 'tlev', 'clwp', 'ciwp', 'h2ovmr',
                    'o3vmr', 'co2vmr', 'ch4vmr', 'n2ovmr', 'o2vmr', 'cfc11vmr',
                    'cfc12vmr', 'cfc22vmr', 'ccl4vmr']:
            assert np.all(self.rrtmg_input[var] >= 0.), f"{var} is insane! Consider debugging..."
        
        relq_msg = ("Effective radius of liq. droplets seems out of valid bounds [2.5, 60] um " +
                    "(or 0 um for cloudfree height layers).")
        assert np.all((self.rrtmg_input['relq'] >= 0.) & (self.rrtmg_input['relq'] <= 60.)), relq_msg
        
        reic_msg = ("Effective radius of ice particles seems out of valid bounds [13, 130] um " +
                    "(or 0 um for cloudfree height layers).")
        assert np.all((self.rrtmg_input['reic'] >= 0.) & (self.rrtmg_input['reic'] <= 130.)), reic_msg
    
    
    def organise_output(self):
        
        self.OUT_DS = xr.Dataset(coords=self.DS.coords)
        
        n_time, n_hgt, n_hgt_h = len(self.DS.time), len(self.DS.height), len(self.DS.height_h)
        for var, flx in self.flxhr.items():
            shape_lay = (n_time, n_hgt)
            shape_lev = (n_time, n_hgt_h)
            
            dims_list = ['time']
            if flx.shape == shape_lay:
                dims_list.append('height')
            elif flx.shape == shape_lev:
                dims_list.append('height_h')
            self.OUT_DS[var] = xr.DataArray(flx, dims=dims_list)
            
            
        # pyRRTMG seems to return incorrect heating rates for the uppermost height layer. 
        # Compute it manually instead:
        spec_hum = h2ovmr_to_q(self.DS.h2o_vmr)
        rho_v = convert_spechum_to_abshum(self.DS.temp, self.DS.pres*100., spec_hum)
        rho = rho_air(self.DS.pres*100., self.DS.temp, rho_v)
        for sky_cond in ['', 'c']:
            for band in ['sw', 'lw']:
                HR_ = compute_heating_rate(self.OUT_DS[f'{band}uflx{sky_cond}'], 
                                           self.OUT_DS[f'{band}dflx{sky_cond}'], 
                                           rho, self.DS.height_h, convert_to_K_day=True)
                self.OUT_DS[f'{band}hr{sky_cond}'] = HR_
        
        
        # information about the variable names and their meanings: 
        # https://github.com/hdeneke/pyRRTMG/blob/master/src_f/_rrtmg.f90 , paragraph "! Output"
        self.OUT_DS['height'].attrs = {'long_name': "Height of the full levels (centre of layer)", 
                                       'units': "m"}
        self.OUT_DS['height_h'].attrs = {'long_name': "Height of the half levels (layer boundaries)", 
                                         'units': "m"}
        self.OUT_DS['swuflx'].attrs = {'long_name': "Upward shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swdflx'].attrs = {'long_name': "Downward shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swdirflx'].attrs = {'long_name': "Direct shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swhr'].attrs = {'long_name': "Shortwave radiative heating rates for layers",
                                       'units': "K day-1"}
        self.OUT_DS['swuflxc'].attrs = {'long_name': "Clear sky upward shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swdflxc'].attrs = {'long_name': "Clear sky downward shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swdirflxc'].attrs = {'long_name': "Clear sky direct shortwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['swhrc'].attrs = {'long_name': "Clear sky shortwave radiative heating rates for layers",
                                       'units': "K day-1"}
        self.OUT_DS['lwuflx'].attrs = {'long_name': "Upward longwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['lwdflx'].attrs = {'long_name': "Downward longwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['lwhr'].attrs = {'long_name': "Longwave radiative heating rates for layers",
                                       'units': "K day-1"}
        self.OUT_DS['lwuflxc'].attrs = {'long_name': "Clear sky upward longwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['lwdflxc'].attrs = {'long_name': "Clear sky downward longwave flux at half level",
                                       'units': "W m-2"}
        self.OUT_DS['lwhrc'].attrs = {'long_name': "Clear sky longwave radiative heating rates for layers",
                                       'units': "K day-1"}
        
        self.OUT_DS.attrs['title'] = "RRTMG outputs from T-CARS environment"
    
    
    def save_output(self, path: str, filename: str):
        
        os.makedirs(path, exist_ok=True)
        
        self.OUT_DS.attrs['history'] = (f"{str(np.datetime64('now')).replace('T', ' ')}, created with " +
                                        f"ex_RRTMG_WS25-26.ipynb;")
        self.OUT_DS = encode_time(self.OUT_DS)
        
        outfile = path + filename
        self.OUT_DS.to_netcdf(outfile, mode='w', format='NETCDF4')
        print(f"Saved {outfile}....")

In [ ]:
def define_cloud(cloudtypes: list):
    
    cloudvars = {'liquid': 'lwp', 'ice': 'iwp'}
    re_cloud = {'lwp': 're_liq', 'iwp': 're_ice'}
    re_value = {'liquid': 5., 'ice': 30.}   # effective radii in um (micrometres)
    
    def fill_cloud_data(
        new_cloud_vars: dict, 
        cloudvar: str, 
        lwp_iwp_lim=500.1, 
        cloud_fraction=1., 
        re_=5.):
        
        new_cloud_vars[cloudvar] = np.arange(0., lwp_iwp_lim, 10.)
        if isinstance(new_cloud_vars[cloudvar], (float, int)):
            new_cloud_vars[cloudvar] = np.array([new_cloud_vars[cloudvar]])
        new_cloud_vars['clc'] = np.full(new_cloud_vars[cloudvar].shape, cloud_fraction)
        new_cloud_vars[re_cloud[cloudvar]] = np.full(new_cloud_vars[cloudvar].shape, re_)
        
        return new_cloud_vars
        
    new_cloud_vars = dict()
    for cloudtype in cloudtypes:
        cloudvar = cloudvars[cloudtype]
        new_cloud_vars = fill_cloud_data(new_cloud_vars, cloudvar, lwp_iwp_lim=500.1, cloud_fraction=1.,
                                         re_=re_value[cloudtype])
        
    if len(cloudtypes) == 0:        # no cloud
        cloudvar = 'lwp'
        new_cloud_vars = fill_cloud_data(new_cloud_vars, cloudvar, lwp_iwp_lim=0.1, cloud_fraction=0.,
                                         re_=0.)
        
    return new_cloud_vars, cloudvar
    
    
def compute_radiative_flux_products(DS: xr.Dataset):
    
    DS = DS.mean('time')
    DS_TOA = DS.isel(height_h=-1)
    DS_SFC = DS.sel(height_h=0., method='nearest')
    
    OUT_DS = xr.Dataset()
    OUT_DS['NET_TOA'] = DS_TOA['swdflx'] - DS_TOA['swuflx'] + DS_TOA['lwdflx'] - DS_TOA['lwuflx']
    OUT_DS['SW_DOWN_TOA'] = DS_TOA['swdflx']
    OUT_DS['SW_UP_TOA'] = DS_TOA['swuflx']
    OUT_DS['LW_DOWN_TOA'] = DS_TOA['lwdflx']
    OUT_DS['LW_UP_TOA'] = DS_TOA['lwuflx']
    
    OUT_DS['NET_SFC'] = DS_SFC['swdflx'] - DS_SFC['swuflx'] + DS_SFC['lwdflx'] - DS_SFC['lwuflx']
    OUT_DS['SW_DOWN_SFC'] = DS_SFC['swdflx']
    OUT_DS['SW_UP_SFC'] = DS_SFC['swuflx']
    OUT_DS['LW_DOWN_SFC'] = DS_SFC['lwdflx']
    OUT_DS['LW_UP_SFC'] = DS_SFC['lwuflx']
    
    OUT_DS['NET_ATM'] = OUT_DS['NET_TOA'] - OUT_DS['NET_SFC']
    
    OUT_DS['DIR_DIFF'] = DS_SFC['swdirflx'] / (DS_SFC['swdflx'] - DS_SFC['swdirflx'])
    
    OUT_DS['TRANS'] = DS_SFC['swdflx'] / DS_TOA['swdflx']
    
    return OUT_DS


def plot_heating_rates(
    DS: xr.Dataset,
    filename="metrad_heating_rates_sw_lw.png",
    path=os.getcwd()+"/plots/",
    save_figures=False):
    
    DS = DS.mean('time')
    
    f1, axs = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(7,5))
    
    plt.subplots_adjust(left=0.125, right=0.96, top=0.96, bottom=0.12, wspace=0.08)
    
    all_sky_style = dict(color='tab:orange', linewidth=linewidth, marker='o', markersize=6)
    clear_sky_style = dict(color='tab:blue', linewidth=linewidth, marker='x', linestyle='dashed', markersize=6)
    axs[0].plot(DS.lwhr, DS.height*0.001, label='All sky', **all_sky_style)
    axs[0].plot(DS.lwhrc, DS.height*0.001, label='Clear sky', **clear_sky_style)
    
    axs[1].plot(DS.swhr, DS.height*0.001, label='All sky', **all_sky_style)
    axs[1].plot(DS.swhrc, DS.height*0.001, label='Clear sky', **clear_sky_style)
    
    for ax in axs:
        lh, ll = ax.get_legend_handles_labels()
        ax.legend(lh, ll, loc='best', frameon=False)
        
        ax.grid(which='major', axis='both', color=(0.5,0.5,0.5), alpha=0.3)
    
    axs[0].set_ylabel("Height (km)")
    axs[0].set_xlabel("Longwave heating rate (K$\,$day$^{-1}$)")
    axs[1].set_xlabel("Shortwave heating rate (K$\,$day$^{-1}$)")
    
    if save_figures:
        os.makedirs(path, exist_ok=True)
        
        plotfile = os.path.join(path, filename)
        f1.savefig(plotfile, dpi=150, bbox_inches='tight')

        print(f"Saved {plotfile} ....")
    else:
        plt.show()
        
    plt.close()


def plot_cloud_forcing_heating_rate(
    DS: xr.Dataset,
    filename="metrad_cloud_forcing_sw_lw.png",
    path=os.getcwd()+"/plots/",
    save_figures=False):
    
    DS = DS.mean('time')
    
    f1 = plt.figure(figsize=(4,4))
    a1 = plt.axes()
    
    plt.subplots_adjust(left=0.15, right=0.96, top=0.96, bottom=0.15)
    
    lw_style = dict(color='tab:blue', linewidth=linewidth, marker='o', markersize=6)
    sw_style = dict(color='tab:orange', linewidth=linewidth, marker='o', markersize=6)
    sum_style = dict(color='k', linewidth=linewidth, marker='o', markersize=6)
    a1.plot(DS.lwhr - DS.lwhrc, DS.height*0.001, **lw_style, label='LW')
    a1.plot(DS.swhr - DS.swhrc, DS.height*0.001, **sw_style, label='SW')
    a1.plot((DS.lwhr - DS.lwhrc) + (DS.swhr - DS.swhrc), DS.height*0.001, **sum_style, label="LW+SW")
    
    lh, ll = a1.get_legend_handles_labels()
    a1.legend(lh, ll, loc='best', frameon=False)
    a1.grid(which='major', axis='both', color=(0.5,0.5,0.5), alpha=0.3)
    
    a1.set_ylabel("Height (km)")
    a1.set_xlabel("Cloud forcing (K$\,$day$^{-1}$)")
    
    if save_figures:
        os.makedirs(path, exist_ok=True)
        
        plotfile = os.path.join(path, filename)
        f1.savefig(plotfile, dpi=150, bbox_inches='tight')

        print(f"Saved {plotfile} ....")
    else:
        plt.show()
        
    plt.close()


def plot_transmission_direct_diffuse_rad_lwp(
    DS: xr.Dataset,
    xaxis_data='lwp',
    filename="metrad_transmission.png",
    path=os.getcwd()+"/plots/",
    save_figures=False):
    
    f1, axs = plt.subplots(nrows=2, ncols=1, sharex=True, figsize=(5,5))
    
    plt.subplots_adjust(left=0.1, right=0.96, top=0.96, bottom=0.12, hspace=0.08)
    
    axs[0].plot(DS[xaxis_data], DS.TRANS*100., linewidth=linewidth)
    axs[1].plot(DS[xaxis_data], DS.DIR_DIFF, linewidth=linewidth)
    
    for ax in axs:
        ax.grid(which='major', axis='both', color=(0.5,0.5,0.5), alpha=0.3)
    
    axs[0].set_ylabel("Transmission (%)")
    axs[1].set_ylabel("Ratio direct/diffuse radiation ( )")
    axs[1].set_xlabel(f"{xaxis_data.upper()} and IWP" + " (g$\,$m$^{-1}$)")
    
    if save_figures:
        os.makedirs(path, exist_ok=True)
        
        plotfile = os.path.join(path, filename)
        f1.savefig(plotfile, dpi=150, bbox_inches='tight')

        print(f"Saved {plotfile} ....")
    else:
        plt.show()
        
    plt.close()

### Changing the atmospheric profile

In order to analyze the effect of trace gases (CO<sub>2</sub>), surface albedo, temperature, humidity and clouds on the radiative fluxes, the input data is modified. This can be done directly in the **metrad_rrtmg_ex.py** cell. There are three possibilities to change the profile:
1. Scaling the whole profile: change the corresponding factor in the `scaling` dictionary.
2. Adding an offset in each layer: change the corresponding values in the `offset` dictionary.
3. Changing directly the profile values and special parameters: Modify variables in the xarray Dataset DS after loading it with `read_default_profiles` or modify `tcars_client.DS`.

If clouds shall be included in the profile, modify the variable `cloudtypes`, which is either an empty list, or a list containing the elements 'liquid', 'ice' or both. Optionally, you can change the effective radius of the liquid or ice cloud and define your own liquid and ice water paths by adjusting the following variables in the function `define_cloud`: `re_value`, `lwp_iwp_lim` (the latter is an argument of the `fill_cloud_data` function). The cloud fraction 'clc' is automatically set to 1 at the same height where liquid water path (LWP) or ice water path (IWP) is added.

Because of the chosen parameterization of the optical properties of liquid clouds (Hu und Stamnes, 1993), `re_value['liquid']` must have a value between 2.5 and 60 μm. For `re_value['ice']`, values between 13 and 130 μm have to be used (Ebert und Curry, 1992). By default, typical values are set for `re_value['liquid']` and `re_value['ice']`: 5 µm and 30 µm, respectively.

### Reading the output and plotting the results

The cell below the **metrad_rrtmg_ex.py** cell executes the RRTMG simulations, as well as some basic plotting and output saving routines. Theoretically, the lines saving the output can be commented out because the results can also be directly plotted using that cell. Default plots are:
- `plot_heating_rates`: Longwave and shortwave heating rates for clear and all sky conditions (note that clear and all sky heating rates are identical if no cloud has been defined)
- `plot_cloud_forcing_heating_rate`: Cloud forcing heating rates for the shortwave and longwave spectrum
- `plot_transmission_direct_diffuse_rad_lwp`: Transmission of shortwave radiative flux and the ratio of direct and diffuse shortwave radiative fluxes

### Tasks and questions

Start with the temperature-/humidity profile of the mid-latitude standard atmosphere in the summer (cosine of solar zenith angle=0.7). Set the solar surface albedo to 0.2. These values are readily set in `read_default_profiles`.

<img src="glob_energy_flows_overview.png" width="600">

### Clear-sky studies

1. Calculate the radiation budget (S↓ - S↑ + L↓ - L↑) at the surface, at the top of the atmosphere (TOA) and for the atmosphere. Note that the computation is already performed using the function `compute_radiative_flux_products`, written to the variable `OUT_DS` in the plotting cell.

For the following exercises, change the settings in the **metrad_rrtmg_ex.py** cell as described below. Make sure that you always undo the changes before you move on to the next exercise so that only one variable is changed for each exercise.

2. Calculate the radiation budget of the atmosphere for CO<sub>2</sub> concentration at pre-industrial times (280 ppm) and four times this value (note that 1. used CO<sub>2</sub> concentrations of 400 ppm). For this, set  
   `tcars_client.iflag_co2_vmr = 1`  
   For quadrupling the CO<sub>2</sub> concentration, additionally change the value of the key `co2_vmr` of the `scaling` dictionary to 4. Determine the difference of both cases and discuss implications for climate and possible feedback mechanisms.
3. What happens if the humidity is increased by 50 % in the whole atmosphere? Change the value of the key `h2o_vmr` of the `scaling` dictionary to 1.5.
4. How does a) an increase of the temperature profile by 10 K and b) an increase of the temperature in the lowest layer by 10 K affect the results?  
   For a), set the value of the key `temp_h` of the `offset` dictionary to 10.  
   For b), add 20 K to the first value in `temp_h`: For this, use  
   `tcars_client.DS['temp_h'][...,0] += 20.`  
   Note that adding 20 K to the lower boundary of the lowest layer (level index 0) while keeping its upper boundary (level index 1) unchanged results in an increase of the layer temperature by 10 K.
5. How does the radiation budget change, if the surface albedo has typical values for ocean (\~0.06), forest (\~0.15), grassland (\~0.2) and desert (\~0.3)? To keep it simple, use the same value for both diffuse and direct albedo. For this, change the value given to  
   `tcars_client.DS[var][:] = `  
   accordingly, where var takes the variable names 'alb_dir_uv', 'alb_dif_uv', 'alb_dir_nir' and 'alb_dif_nir' (see surrounding for loop).

### Cloudy-sky sensitivities

Change the albedo back to 0.2.
Add a liquid water cloud to the standard atmosphere between 2 and 3 km. Set  
`cloudtypes = ['liquid']`  
and make sure that  
`cloud_height = np.array([2000., 3000.])`

By default, these RRTMG simulations are set up to simulate liquid water path (LWP) values between 0 and 500 g m<sup>-2</sup> with a step of 10 g m<sup>-2</sup> (see function `fill_cloud_data`in `define_cloud`). Plots are only created for LWP 0, 30, 100 and 500 g m<sup>-2</sup>. The cloud cover `clc` and droplet effective radii `re_liq` are automatically set to 1 and typical values, respectively, where needed.

6. How does the radiation budget change?
7. Depict the transmission of the solar radiation as a function of the total liquid water path of the cloud (LWP). You can use `plot_transmission_direct_diffuse_rad_lwp` for this. This function is executed automatically by default.
8. How does the ratio direct/diffuse radiation change if the LWP is increased? For visualisation, you can also use `plot_transmission_direct_diffuse_rad_lwp`.
9. What happens if the cloud (LWP: 100 g m<sup>-2</sup>) is vertically moved and located between 5 and 6 km height? Adapt the variable `cloud_height`. You can use the variable `OUT_DS_all`, which contains all needed variables for all simulated LWP values (default: 0 to 500 g m<sup>-2</sup>in 10 g m<sup>-2</sup> steps).
10. What happens if the cloud from 9. is replaced by an ice cloud with the same water content? Set:  
    `cloudtypes = ['ice']`
11. And if both liquid and ice clouds occur between 5 and 6 km? Use:  
    `cloudtypes = ['liquid', 'ice']`

### Heating rate studies

12. Plot the shortwave and longwave heating rate profiles for the six atmospheric states in 6., 9., 10. and 11.. Always plot both profiles, with clouds and clear sky. For plotting, you can use `plot_heating_rates`.
13. Plot also the difference of cloud and clear sky heating rates, i.e. the cloud radiative forcing. Plot the cloud radiative forcing for the SW, the LW and the net value (SUM of SW and LW). You can use `plot_cloud_forcing_heating_rate`.
14. Which state leads particularly to a warming/cooling of the atmosphere?


In [ ]:
# metrad_rrtmg_ex.py

path_data = {'tcars_data': "/data/metrad/tcars/",  # should hopefully work like this, otherwise, Andreas Walbröl will provide the data
             }
path_output = os.getcwd() + "/rrtmg_output/"
path_plots = os.getcwd() + "/plots/"

set_dict = {'save_figures': True,       # if True: plots are only saved to file; if False. plots are shown here
            }

# scaling: to scale the parameters in read_default_profiles
scaling = dict(
      pres      = 1.,
      pres_h    = 1.,
      temp_h    = 1.,
      lwp       = 1.e+00,
      iwp       = 1.e+00,
      re_liq    = 1.,
      re_ice    = 1.,
      h2o_vmr   = 1.,           # modify for question 03
      co2_vmr   = 1.,           # modify for question 02
      o3_vmr    = 1.,
      n2o_vmr   = 1.,
      ch4_vmr   = 1.,
      o2_vmr    = 1.,
      height    = 1.,
      height_h  = 1.,
      )

# offset: to apply additive offsets to the data in read_default_profiles
offset = dict(
      pres       = 0.,
      pres_h     = 0.,
      temp_h     = 0.,          # modify for question 04
      lwp        = 0.,
      iwp        = 0.,
      re_liq     = 0.,
      re_ice     = 0.,
      h2o_vmr    = 0.,
      co2_vmr    = 0.,
      o3_vmr     = 0.,
      n2o_vmr    = 0.,
      ch4_vmr    = 0.,
      o2_vmr     = 0.,
      height     = 0.,
      height_h   = 0.,  
      )


DS = read_default_profiles(scaling=scaling, offset=offset)    
tcars_client = tcars(path_tcars_data=path_data['tcars_data'], DS=DS)

# filename suffixes:
suffix = "_q02"         # REMEMBER TO RENAME YOUR OUTPUT !!
subtask = "a"            # e.g., "a", "b", ...

# DATA MODIFICATIONS:
tcars_client.iflag_co2_vmr = 1             # modify for question 02; (for valid values, see tcars.py.__init__)
tcars_client.DS['temp_h'][...,0] += 0.     # modify for question 04b
for var in ['alb_dir_uv', 'alb_dif_uv', 'alb_dir_nir', 'alb_dif_nir']:
    tcars_client.DS[var][:] = 0.2          # modify for question 05

cloudtypes = []     # for questions 6-9, use ['liquid']
                    # for question 10: use ['ice']
                    # for question 11: use ['liquid', 'ice']
                    # for questions 12-14: use whatever required of the task
cloud_height = np.array([2000., 3000.])    # lower and upper limit of cloud layer; modify accordingly for questions 06 - 14
new_cloud_vars, cloudvar = define_cloud(cloudtypes)


Modify the cell below if you would like to change the output (e.g., define new plots, compute other stuff, ...).

In [ ]:
OUT_DS_list = list()
count = 0

for k in tqdm(range(len(new_cloud_vars[cloudvar]))):
    
    for key, var in new_cloud_vars.items():
        tcars_client.DS[key].loc[{'height': slice(cloud_height[0],cloud_height[1])}][...] = var[k]
    
    
    tcars_client.set_rrtmg_input()
    tcars_client.run_tcars()         # actually runs the radiative transfer simulations; output is put into tcars_client.OUT_DS

    
    OUT_DS = compute_radiative_flux_products(tcars_client.OUT_DS)
    OUT_DS_list.append(OUT_DS)
    
    if np.any(np.abs(new_cloud_vars[cloudvar][k] - np.array([0., 30., 100., 500.])) == 0.):
        if len(new_cloud_vars[cloudvar]) > 1: subtask = panel_marker[count]
        print(f"TASK{suffix}{subtask}\nBudget surface: {OUT_DS.NET_SFC:.1f} W m-2\n" +
              f"Budget TOA: {OUT_DS.NET_TOA:.1f} W m-2\nBudget atmosphere: {OUT_DS.NET_ATM:.1f} W m-2\n")

        
        plot_heating_rates(tcars_client.OUT_DS, 
                           filename=f"metrad_heating_rates_sw_lw{suffix}{subtask}.png",
                           path=path_plots, 
                           **set_dict)
        plot_cloud_forcing_heating_rate(tcars_client.OUT_DS,
                                        filename=f"metrad_cloud_forcing_sw_lw{suffix}{subtask}.png",
                                        path=path_plots, 
                                        **set_dict)
        
        tcars_client.save_output(path=path_output, filename=f"metrad_rrtmg{suffix}{subtask}.nc")
        OUT_DS.to_netcdf(path_output + f"metrad_radiation_budget{suffix}{subtask}.nc", mode='w', format='NETCDF4')
        
        count += 1

OUT_DS_all = xr.concat(OUT_DS_list, dim=cloudvar).assign_coords({cloudvar: ([cloudvar], new_cloud_vars[cloudvar])})
if len(OUT_DS_all[cloudvar]) > 1:
    plot_transmission_direct_diffuse_rad_lwp(OUT_DS_all, 
                                             xaxis_data=cloudvar,
                                             filename=f"metrad_transmission{suffix}.png",
                                             path=path_plots,
                                             **set_dict)